In [1]:
import math
import utils
import torch
import pandas as pd
import validationFunctions as vF
from torch.utils.data import TensorDataset, DataLoader

pathTrain:str = "CUP/ML-CUP23-TRAIN.csv"
seed:int = 21
## TRAIN IMPORT DATASET
dataset = utils.importDatasetCup(pathTrain)
dataset_train, dataset_test = vF.train_test_split(dataset=dataset)
# CREATE KFOLD
dataset_train, dataset_val = vF.Kfold(dataset_train)
#print(self.dataset_train)
#print(self.dataset_val)
# SPLIT FOLDS
x_train, x_test, y_train, y_test = vF.split_folds(dataset_val, dataset_train) 


## CONVERT TO TENSOR TRAIN SET
for i in range(4):
    x_train[i] = torch.tensor(x_train[i].to_numpy())
    y_train[i] = torch.tensor(y_train[i].to_numpy())
    # SET TYPE DOUBLE
    x_train[i] = x_train[i].double()
    y_train[i] = y_train[i].double()
    # CONVERT TO TENSOR
    x_test[i] = torch.tensor(x_test[i].to_numpy())
    y_test[i] = torch.tensor(y_test[i].to_numpy())
    # SET TYPE DOUBLE
    x_test[i] = x_test[i].double()
    y_test[i] = y_test[i].double()

for kfold in range(4):
    # CREATE DATALOADER TRAIN
    batchTrain =  600
    print("Batch size for training: ", batchTrain)
    dataset_train = TensorDataset(x_train[kfold], y_train[kfold])
    data_loader_train = DataLoader(dataset_train, batch_size=batchTrain, shuffle=False)
    # CREATE DATALOADER TEST
    batchTest = 200
    print("Batch size for testing: ", batchTest)
    dataset_test = TensorDataset(x_test[kfold], y_test[kfold])
    data_loader_test = DataLoader(dataset_test, batch_size=batchTest, shuffle=False)
    
    for batch_input, batch_output in data_loader_test:
        print(batch_input)
        print(batch_output)

Batch size for training:  600
Batch size for testing:  200
tensor([[-0.9276,  0.9961, -0.9975,  ...,  0.9848, -0.3451, -0.5299],
        [-0.8594,  0.9307, -0.9928,  ...,  0.9730, -0.9246,  0.1183],
        [-0.9846,  0.9944, -0.9988,  ...,  0.9177, -0.1641,  0.3726],
        ...,
        [ 0.9877,  0.1233,  0.8074,  ...,  0.9945, -0.9549, -0.9664],
        [-0.7745,  0.5851, -0.9918,  ...,  0.9377, -0.9568,  0.6304],
        [-0.3942, -0.7153, -0.9777,  ...,  0.8705, -0.9719,  0.6980]],
       dtype=torch.float64)
tensor([[-2.5083e+00,  1.4212e+01,  1.8799e+01],
        [ 4.9790e+00,  5.2067e+00,  3.1624e+01],
        [ 5.2688e+00,  9.3186e+00,  1.3542e+01],
        [-2.3257e+01,  1.1464e+01,  5.7568e+00],
        [-6.3127e+01, -2.2198e+01,  2.3113e+01],
        [-2.9367e+01,  1.4535e+00,  1.1981e+01],
        [ 2.5873e+00,  5.3924e+00,  6.6251e+00],
        [-8.2480e+00,  1.3443e+01,  5.9878e+00],
        [ 6.8893e-01,  1.2460e+01,  2.5537e+01],
        [-7.0495e+01, -3.0570e+00,  4.